In [ ]:
import os
import cv2
from ultralytics import YOLO
import matplotlib.pyplot as plt
import numpy as np
import albumentations as A
from tqdm import tqdm

## Разметка с тяжелой YOLO

In [ ]:
def annotate_images(images_dir, output_dir, classes_to_label, model_weights_path, conf_threshold=0.25):

    img_out_dir = os.path.join(output_dir, "imgages")
    labels_out_dir = os.path.join(output_dir, "labels")
    os.makedirs(img_out_dir, exist_ok=True)
    os.makedirs(labels_out_dir, exist_ok=True)

    model = YOLO(model_weights_path)

    model_classes = model.model.names  # id: name

    # Define the classes for labeling
    if all(isinstance(c, str) for c in classes_to_label):
        class_ids = [i for i, name in model_classes.items() if name in classes_to_label]
    else:
        class_ids = classes_to_label

    # Read data
    for fname in os.listdir(images_dir):
        if not fname.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp')):
            continue
        img_path = os.path.join(images_dir, fname)
        img = cv2.imread(img_path)
        img_vis = img.copy()
        if img is None:
            continue

        results = model(img, conf=conf_threshold)[0]

        # YOLO label 
        label_lines = []
        for box in results.boxes:
            cls_id = int(box.cls[0])
            if cls_id not in class_ids:
                continue

            x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
            h, w = img.shape[:2]
            x_center = ((x1 + x2) / 2) / w
            y_center = ((y1 + y2) / 2) / h
            width = (x2 - x1) / w
            height = (y2 - y1) / h
            label_lines.append(f"{cls_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}")

            x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])
            color = (0, 255, 0)

            cv2.rectangle(img_vis, (x1, y1), (x2, y2), color, 2)
            # Check classes
            label = model_classes[cls_id] if isinstance(model_classes, dict) else str(cls_id)
            cv2.putText(img_vis, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)

        cv2.imshow("Detected", img_vis)
        cv2.waitKey(0)        

        # Save img
        cv2.imwrite(os.path.join(img_out_dir, fname), img)

        # Save label
        label_fname = os.path.splitext(fname)[0] + ".txt"
        with open(os.path.join(labels_out_dir, label_fname), "w") as f:
            f.write("\n".join(label_lines))


In [ ]:
annotate_images(
    images_dir="C:\\Users\\Alex\\Desktop\\dataset",
    output_dir="data_labeled",
    classes_to_label=[40, 41, 42, 46, 47, 48, 49, 50, 51,52, 53, 54, 55, 56],  # Nost of related to food
    model_weights_path="yolo11x.pt"
)

## Заменяем классы по COCO на новые наши

In [ ]:
def remap_yolo_classes_in_labels(labels_dir, class_id_map):
    """
    labels_dir: path to dataset/labels with .txt
    class_id_map: dict, key - old id, value — new id ({45:0, 54:1, 78:2})
    """
    for fname in os.listdir(labels_dir):
        if not fname.endswith('.txt'):
            continue
        file_path = os.path.join(labels_dir, fname)
        with open(file_path, 'r') as f:
            lines = f.readlines()
        new_lines = []
        changed = False
        for line in lines:
            parts = line.strip().split()
            if not parts:
                continue
            try:
                class_id = int(parts[0])
            except ValueError:
                new_lines.append(line)
                continue
            if class_id in class_id_map:
                parts[0] = str(class_id_map[class_id])
                changed = True
            new_lines.append(' '.join(parts) + '\n')
        if changed:
            with open(file_path, 'w') as f:
                f.writelines(new_lines)

In [ ]:
remap_yolo_classes_in_labels("C:\\Users\\Alex\\Desktop\\dataset_food\\labels", {46:0})

In [ ]:
input_images_dir = "C:\\Users\\Alex\\Desktop\\food_det_dataset\\train\\images"
input_labels_dir = "C:\\Users\\Alex\\Desktop\\food_det_dataset\\train\\labels"
output_dir = "C:\\Users\\Alex\\Desktop\\food_det_dataset\\train_aug2"
os.makedirs(f"{output_dir}/images", exist_ok=True)
os.makedirs(f"{output_dir}/labels", exist_ok=True)

# Аугментации
augmentation = A.Compose([
    A.VerticalFlips(p=0.5), 
    A.MotionBlur(blur_limit=(3, 7), p=0.4),
    A.RandomBrightnessContrast(p=0.4),
    A.CoarseDropout(max_holes=4, max_height=0.05, max_width=0.05, p=0.6)
], bbox_params=A.BboxParams(format='yolo', min_visibility=0.3))

def read_yolo_label(label_path, img_width, img_height):
    if not os.path.exists(label_path):
        return []
    with open(label_path, 'r') as f:
        lines = f.readlines()
    bboxes = []
    for line in lines:
        parts = list(map(float, line.strip().split()))
        if len(parts) == 5:
            class_id, x_center, y_center, width, height = parts
            
            bboxes.append([x_center, y_center, width, height, class_id])
    return bboxes

def write_yolo_label(label_path, bboxes):
    with open(label_path, 'w') as f:
        for bbox in bboxes:
            if len(bbox) == 5:  # [x,y,w,h,class_id]
                class_id, x_center, y_center, width, height = bbox[4], bbox[0], bbox[1], bbox[2], bbox[3]
                f.write(f"{int(class_id)} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n")


image_files = [f for f in os.listdir(input_images_dir) if f.endswith(('.jpg', '.png'))]
print(f"Найдено {len(image_files)} изображений.")

for image_file in tqdm(image_files):
    image_path = os.path.join(input_images_dir, image_file)
    label_path = os.path.join(input_labels_dir, os.path.splitext(image_file)[0] + '.txt')
    
    image = cv2.imread(image_path)
    if image is None:
        print(f"Ошибка чтения: {image_path}")
        continue
    
    img_height, img_width = image.shape[:2]
    original_bboxes = read_yolo_label(label_path, img_width, img_height)
    
    if not original_bboxes:
        print(f"Нет bboxes для {image_file}")
        continue
    
    # Augment 3 times
    for aug_idx in range(3):
        try:
            bboxes_for_aug = [bbox[:4] for bbox in original_bboxes]
            class_ids = [bbox[4] for bbox in original_bboxes]
            
            augmented = augmentation(image=image, bboxes=bboxes_for_aug)
            aug_image = augmented['image']
            aug_bboxes = augmented['bboxes']
            
            if not aug_bboxes:
                print(f"Все bboxes отброшены для {image_file}_aug{aug_idx}")
                continue
            
            #  class id list
            final_bboxes = []
            for i, bbox in enumerate(aug_bboxes):
                if i < len(class_ids):  
                    final_bboxes.append([*bbox, class_ids[i]])
            
            aug_image_name = f"{os.path.splitext(image_file)[0]}_aug{aug_idx}.jpg"
            aug_image_path = os.path.join(output_dir, "images", aug_image_name)
            cv2.imwrite(aug_image_path, aug_image)
            
            aug_label_name = f"{os.path.splitext(image_file)[0]}_aug{aug_idx}.txt"
            aug_label_path = os.path.join(output_dir, "labels", aug_label_name)
            write_yolo_label(aug_label_path, final_bboxes)
            
        except Exception as e:
            print(f"Ошибка при аугментации {image_file}: {e}")

print("Аугментация завершена! Результаты в:", output_dir)

C:\Users\Alex\AppData\Local\Temp\ipykernel_37524\4131152067.py:19: UserWarning: Argument(s) 'max_holes, max_height, max_width' are not valid for transform CoarseDropout
  A.CoarseDropout(max_holes=4, max_height=0.05, max_width=0.05, p=0.6)


Найдено 160 изображений.


 79%|███████▉  | 127/160 [00:10<00:02, 13.29it/s]

Все bboxes отброшены для frame_0073_jpg.rf.01cc1688da10effe3813274cf9bb551f.jpg_aug0


100%|██████████| 160/160 [00:12<00:00, 12.63it/s]

Аугментация завершена! Результаты в: C:\Users\Alex\Desktop\food_det_dataset\train_aug2
